In [33]:
#import dependencies
import pandas as pd
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import datetime as dt
import sqlite3
import numpy as np

In [5]:
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [6]:
database_path = "Data/stock_data_db.db"

In [7]:
URI = f"sqlite:///{database_path}"

In [8]:
engine = create_engine(URI)
conn = engine.connect()

In [9]:
symbol_company_table=pd.read_sql("SELECT * FROM symbol_company", conn)

In [10]:
print(symbol_company_table.groupby('Industry').count())

                           field1  Symbol  CompanyName  MarketCap
Industry                                                         
Advertising Agencies           31      31           31         31
Aerospace & Defense            54      54           54         54
Agricultural Inputs            18      18           18         18
Airlines                       20      20           20         20
Airports & Air Services        11      11           11         11
...                           ...     ...          ...        ...
Utilities-Regulated Gas        13      13           13         13
Utilities-Regulated Water      14      14           14         14
Utilities-Renewable            14      14           14         14
Waste Management               20      20           20         20
n/a                             8       8            8          8

[152 rows x 4 columns]


In [11]:
data_table = pd.read_sql("SELECT * FROM full_stock_df", conn)
data_table

,Date,Open,High,Low,Close,AdjClose,Volume,Name,CompanyName
0,1999-12-31,56.866955,57.179901,54.542202,55.302216,47.680683,1931197.0,A,Agilent Technologies
1,2000-01-03,56.330471,56.464592,48.193848,51.502148,44.404324,4674353.0,A,Agilent Technologies
2,2000-01-04,48.730328,49.266811,46.316166,47.567955,41.012325,4765083.0,A,Agilent Technologies
3,2000-01-05,47.389126,47.567955,43.141991,44.617310,38.468330,5758642.0,A,Agilent Technologies
4,2000-01-06,44.080830,44.349072,41.577251,42.918453,37.003593,2534434.0,A,Agilent Technologies
...,...,...,...,...,...,...,...,...,...
16241142,2021-08-13,14.210000,14.290000,13.860000,14.190000,14.190000,287300.0,ZYXI,Zynex
16241143,2021-08-16,14.150000,14.180000,13.560000,13.650000,13.650000,284100.0,ZYXI,Zynex
16241144,2021-08-17,13.550000,13.640000,12.950000,13.200000,13.200000,386200.0,ZYXI,Zynex
16241145,2021-08-18,13.270000,13.310000,12.680000,13.150000,13.150000,316900.0,ZYXI,Zynex


In [14]:

data_table.Date = pd.to_datetime(data_table.Date)


In [16]:
#add new columns D, M, Y to make filtering and calculations easier
data_table['day'] = data_table['Date'].dt.day
data_table['month'] = data_table['Date'].dt.month
data_table['year'] = data_table['Date'].dt.year

In [17]:
data_table.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume', 'Name',
       'CompanyName', 'day', 'month', 'year'],
      dtype='object')

In [22]:
#filter our dataframe to only data within the last 10 years to reduce runtime lag
last_decade_df = data_table.loc[(data_table['year']>2010)]
last_decade_df

,Date,Open,High,Low,Close,AdjClose,Volume,Name,CompanyName,day,month,year
2768,2011-01-03,29.728184,30.143063,29.620888,29.957083,27.412437,4994075.0,A,Agilent Technologies,3,1,2011
2769,2011-01-04,30.035765,30.114449,29.456366,29.678112,27.157160,5017282.0,A,Agilent Technologies,4,1,2011
2770,2011-01-05,29.513592,29.849785,29.327612,29.613733,27.098249,4519035.0,A,Agilent Technologies,5,1,2011
2771,2011-01-06,29.592276,29.928469,29.477825,29.670959,27.150610,4699097.0,A,Agilent Technologies,6,1,2011
2772,2011-01-07,29.699572,29.899857,29.356222,29.771103,27.242252,3810948.0,A,Agilent Technologies,7,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...
16241142,2021-08-13,14.210000,14.290000,13.860000,14.190000,14.190000,287300.0,ZYXI,Zynex,13,8,2021
16241143,2021-08-16,14.150000,14.180000,13.560000,13.650000,13.650000,284100.0,ZYXI,Zynex,16,8,2021
16241144,2021-08-17,13.550000,13.640000,12.950000,13.200000,13.200000,386200.0,ZYXI,Zynex,17,8,2021
16241145,2021-08-18,13.270000,13.310000,12.680000,13.150000,13.150000,316900.0,ZYXI,Zynex,18,8,2021


In [37]:
#group by each stock to begin calculations and EDA 
last_decade_grouped = last_decade_df.groupby
last_decade_grouped.head()

UnsupportedFunctionCall: numpy operations are not valid with groupby. Use .groupby(...).mean() instead